In [1]:
import azureml
import json

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment
from azureml.core.dataset import Dataset

import tensorflow as tf
from IPython.display import Image, display

import matplotlib.pyplot as plt
import tensorflow_hub as hub

import numpy as np

import warnings
warnings.filterwarnings('ignore')

2022-09-14 11:41:13.134096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="d6233897-5c9f-47f9-8507-6d4ada2d5183")
resource_group = os.getenv("RESOURCE_GROUP", default="RG_Jaganlal")
workspace_name = os.getenv("WORKSPACE_NAME", default="beanleaf_classifier")
workspace_region = os.getenv("WORKSPACE_REGION", default="centralus")
cluster_name = os.getenv("CLUSTER_NAME", default="beanleaf-classifier-cluster")
dataset_path_in_datastore = os.getenv("BLOB_STORAGE_PATH", default="beanleaf")
data_local_directory = os.getenv("DATA_LOCAL_DIRECTORY", default="../0_data/beanleaf_dataset/")
experiment_name = os.getenv("EXPERIMENT_NAME", default="beanleaf_classifier_experiment")
blob_container = os.getenv("BLOB_CONTAINER", default="beanleaf_dataset")

In [8]:
# ws = Workspace.from_config()
ws = Workspace.get(name=workspace_name,
                    subscription_id=subscription_id,
                    resource_group=resource_group)
ws

Workspace.create(name='beanleaf_classifier', subscription_id='d6233897-5c9f-47f9-8507-6d4ada2d5183', resource_group='RG_Jaganlal')

In [10]:
from azureml.core import ComputeTarget
target = ComputeTarget(ws, cluster_name)
target

AmlCompute(workspace=Workspace.create(name='beanleaf_classifier', subscription_id='d6233897-5c9f-47f9-8507-6d4ada2d5183', resource_group='RG_Jaganlal'), name=beanleaf-classifier-cluster, id=/subscriptions/d6233897-5c9f-47f9-8507-6d4ada2d5183/resourceGroups/RG_Jaganlal/providers/Microsoft.MachineLearningServices/workspaces/beanleaf_classifier/computes/beanleaf-classifier-cluster, type=AmlCompute, provisioning_state=Succeeded, location=centralus, tags={})

# Connect to Workspace

In [5]:
# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print("Compute Target:", compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:", ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name=experiment_name)
print("Experiment:", exp.name)

Compute Target: beanleaf-classifier-cluster
Datastore: workspaceblobstore
Experiment: BeanClassifierExperiment


In [ ]:
from azureml.data.datapath import DataPath

# Upload data to AzureML Datastore
ds = ws.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=data_local_directory,
            target=DataPath(ds, 'beanleaf_dataset'),
            show_progress=True, overwrite=True)

# Register file dataset with AzureML
ds = ds.register(workspace=ws, name="beanleaf", description="Bean leaves with train, test and validation folders", create_new_version=True)
print(f'Dataset {ds.name} registered.')